In [2]:
import Pkg; 
cd(joinpath(@__DIR__, "../../"))
Pkg.activate("Project.toml")

using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [14]:
@load "../../Data/jld2/2_experimental/2_6r7m_protor.jld2"

3-element Vector{Symbol}:
 :template_mol
 :template_radii
 :x_init

In [6]:
symbols == [:template_mol, :template_radii, :x_init]

3×1206 Matrix{Float64}:
 251.173  250.097  249.674  250.396  …  253.216  253.956  253.839  255.427
 197.372  196.396  196.183  196.544     198.527  197.232  196.854  197.243
 191.043  191.139  192.585  193.51      192.311  194.907  196.283  194.518

In [37]:
function load_jld2_and_translate(filepath::String)
    if split(filepath, ".")[end] != "jld2"
        println("No jld2 file: ", filepath)
        return
    end
    root = join(split(filepath, "/")[1:end-1], "/") * "/"
    file = split(filepath, "/")[end]
    data = load(filepath)
    if issetequal(Set(keys(data)),  Set(["template_mol", "template_radii", "x_init"]))
        mkpath(root * "old_format_backup/")
        template_centers = data["template_mol"]
        mv(filepath, "$(root)old_format_backup/$(file)")
        @save filepath template_centers template_radii x_init
    else
        println("The following format has not been parsed yet: ")
        println(keys(data))
    end
end


load_jld2_and_translate (generic function with 1 method)

In [38]:
folder = "../../Data/jld2/2_experimental_test/"
for file in readdir(folder)
    load_jld2_and_translate("$(folder)$(file)")
end

No jld2 file: ../../Data/jld2/2_experimental_test/.DS_Store
